In [1]:
import sys 
import os
from sympy.physics.secondquant import *
from sympy import symbols,I
import sympy as sp


In [2]:
from sympy import symbols, Matrix
n = 6
# Define the symbolic variables for the matrix elements
phi_s = symbols('phi_s')
phi_sij = symbols('phi_sij', cls=symbols)

# Initialize the matrix with zeros
Vs_symbolic = Matrix.zeros(n)

# Define the symbolic variables for the matrix elements
phi_sij = sp.symbols('phi_s:6:6')

# Fill the diagonal and first two upper diagonal elements with specific symbols
for i in range(n):
    Vs_symbolic[i, i] = phi_sij[i * n + i]
    if i < n - 1:
        Vs_symbolic[i, i + 1] = phi_sij[i * n + i + 1]
        Vs_symbolic[i + 1, i] = phi_sij[i * n + i + 1]  # Ensure symmetry
    if i < n - 2:
        Vs_symbolic[i, i + 2] = phi_sij[i * n + i + 2]
        Vs_symbolic[i + 2, i] = phi_sij[i * n + i + 2]  # Ensure symmetry

Vs = Vs_symbolic


In [3]:
Vs

Matrix([
[phi_s00, phi_s01, phi_s02,       0,       0,       0],
[phi_s01, phi_s11, phi_s12, phi_s13,       0,       0],
[phi_s02, phi_s12, phi_s22, phi_s23, phi_s24,       0],
[      0, phi_s13, phi_s23, phi_s33, phi_s34, phi_s35],
[      0,       0, phi_s24, phi_s34, phi_s44, phi_s45],
[      0,       0,       0, phi_s35, phi_s45, phi_s55]])

In [4]:
import sympy as sp
omega_s = sp.Symbol('omega_s')
alpha = sp.Symbol('alpha')
# Generate the diagonal elements
diagonal = [i*omega_s + alpha*i*(i-1)/2 for i in range(0, n)]

# Create the diagonal matrix
Hs = sp.diag(*diagonal)
def generate_H1_H2(d):
    # Create symbolic variables for epsilon1 and epsilon2, but set the first element to 0
    epsilon1 = [0] + [sp.Symbol(f'epsilon^1_{i}') for i in range(1, d)]
    epsilon2 = [0] + [sp.Symbol(f'epsilon^2_{i}') for i in range(1, d)]

    # Generate H1 (diagonal matrix with epsilon^1_i, first element is 0)
    H1 = sp.diag(*epsilon1)

    # Generate H2 (diagonal matrix with epsilon^2_i, first element is 0)
    H2 = sp.diag(*epsilon2)

    return H1, H2


def generate_V1_V2(n):
    # Create symbolic variables for V1
    V1 = sp.zeros(n, n)
    for i in range(n):
        for j in range(n):
            if i != j:
                # Check if both i and j are either both even or both odd
                if (i % 2 == j % 2):
                    V1[i, j] = 0  # Set to 0 if both are odd or both are even
                else:
                    V1[i, j] = sp.Symbol(f'phi^1_{i}{j}')
                    V1[j, i] = V1[i, j]  # Ensure symmetry

    # Create symbolic variables for V2
    V2 = sp.zeros(n, n)
    for i in range(n):
        for j in range(n):
            if i != j:
                # Check if both i and j are either both even or both odd
                if (i % 2 == j % 2):
                    V2[i, j] = 0  # Set to 0 if both are odd or both are even
                else:
                    V2[i, j] = sp.Symbol(f'phi^2_{i}{j}')
                    V2[j, i] = V2[i, j]  # Ensure symmetry

    return V1, V2


# Set the dimension
d = 2

# Generate H1 and H2
H1, H2 = generate_H1_H2(d)
V1, V2 = generate_V1_V2(d)

Is = sp.eye(n)
I1 = I2 = sp.eye(d)


In [5]:
from sympy.physics.quantum import TensorProduct
EL = sp.Symbol('E_L')
phis = sp.Symbol('phi_s')
Hs = TensorProduct(Hs,TensorProduct(I2,I1))
H1 = TensorProduct(Is,TensorProduct(I2,H1))
H2 = TensorProduct(Is,TensorProduct(H2,I1))
H_int1 = -EL/2*TensorProduct(Vs,TensorProduct(I2,V1))
H_int2 = EL/2*TensorProduct(Vs,TensorProduct(V2,I1))


In [6]:
V1

Matrix([
[       0, phi^1_10],
[phi^1_10,        0]])

In [7]:
H0 = Hs + H1 + H2
V = H_int1 + H_int2

In [8]:
import sympy as sp

def shuffle_H0_and_V(H0, V, d):
    # Make H0 mutable to allow modifications
    H0_new = H0.as_mutable()
    
    # Make V mutable to allow modifications
    V_new = V.as_mutable()

        
    # Swap diagonal elements at indices 3 and d+1
    H0_new[3, 3], H0_new[d+1, d+1] = H0[d+1, d+1], H0[3, 3]
    
    # Swap rows and columns at indices 3 and d+1 in V
    V_new[3, :], V_new[d+1, :] = V[d+1, :], V[3, :]
    V_new[:, 3], V_new[:, d+1] = V[:, d+1], V[:, 3]
    
    # Swap diagonal elements at indices 2 and d
    H0_new[2, 2], H0_new[d, d] = H0[d, d], H0[2, 2]
    
    # Swap rows and columns at indices 2 and d in V
    V_new[2, :], V_new[d, :] = V_new[d, :], V_new[2, :]
    V_new[:, 2], V_new[:, d] = V_new[:, d], V_new[:, 2]

    
    # Optionally convert back to immutable matrices if needed
    H0_new = H0_new.as_immutable()
    V_new = V_new.as_immutable()

    return H0_new, V_new
import sympy as sp

def commutator(A, B):
    return A * B - B * A

def nested_commutator(A, B, n):
    if n == 1:
        return commutator(A, B)
    else:
        return commutator(A, nested_commutator(A, B, n-1))

def create_subspace_projector(dim, subspace_indices):
    """
    Create a projector for the given subspace using SymPy.

    Args:
        dim (int): The total dimension of the Hilbert space.
        subspace_indices (list): The indices of the subspace.

    Returns:
        tuple: The projector P for the given subspace and the complementary projector Q (both SymPy matrices).
    """
    # Initialize the projector matrix P as a zero matrix of size dim x dim
    P = sp.zeros(dim, dim)
    
    # Create the basis states and compute the projector for each
    for idx in subspace_indices:
        # Create a basis state (column vector) with 1 at the index 'idx'
        state = sp.zeros(dim, 1)
        state[idx, 0] = 1
        
        # Compute the rank-1 projector for this basis state: |state><state|
        P += state * state.T  # Outer product in SymPy is `*`
    # The complementary projector Q = I - P
    Q = sp.eye(dim) - P
    
    return P, Q

def swt_subspace(H0, V, subspace_indices):
    """
    Compute the Schrieffer-Wolff transformation on a subspace of the Hamiltonian.

    Args:
        H0 (sp.Matrix): The unperturbed Hamiltonian.
        V (sp.Matrix): The perturbation Hamiltonian.
        subspace_indices (list): The indices of the subspace to consider.

    Returns:
        tuple: The Schrieffer-Wolff transformation operators (S1, S2, S3) and 
               the transformed Hamiltonian components (H1, H2, H3, H4).
    """
    dim = H0.shape[0]
    P, Q = create_subspace_projector(dim, subspace_indices)
    
    # Extract the subspace of H0 and V
    Vd = P * V * P + Q * V * Q
    Vod = P * V * Q + Q * V * P

    # Compute the energy differences in the subspace
    delta = sp.Matrix([[H0[i,i] - H0[j,j] if i != j else 1 for j in range(dim)] for i in range(dim)])

    # Compute the Schrieffer-Wolff transformation components
    H1 = Vd
    S1 = sp.Matrix([[Vod[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    H2 = sp.Rational(1, 2) * commutator(S1, Vod)
    S2 = sp.Matrix([[-commutator(Vd, S1)[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])

    # H3 = sp.Rational(1, 2) * commutator(S2, Vod)
    # S3 = sp.Matrix([[(commutator(S2, Vd)[i,j] + sp.Rational(1, 3) * nested_commutator(S1, Vod, 2)[i,j]) / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H4 = sp.Rational(1, 2) * commutator(S3, Vod) - sp.Rational(1, 24) * nested_commutator(S1, Vod, 3)

    return [S1,S2], [H1, H2,]


In [9]:
H0,V = shuffle_H0_and_V(H0, V, d)

In [10]:
indices = [0,1,2,3]
result = swt_subspace(H0, V, indices)

In [11]:
S1,S2 = result[0]

In [15]:
# Initialize chargeop as a zero matrix
chargeop = sp.zeros(n, n)

# Fill in the subdiagonal elements
for i in range(n):
    if i < n - 1:
        chargeop[i, i + 1] = I * sp.Symbol(f'n_{i}{i + 1}')
        chargeop[i + 1, i] = -I * sp.Symbol(f'n_{i}{i + 1}')  # Ensure symmetry
    if i < n - 2:
        chargeop[i, i + 2] = I * sp.Symbol(f'n_{i}{i + 2}')
        chargeop[i + 2, i] = -I * sp.Symbol(f'n_{i}{i + 2}')  # Ensure symmetry

# Apply the tensor product and other operations
chargeop = TensorProduct(chargeop, TensorProduct(I2, I1))



In [16]:

# Define the commutator function
def comm(X, Y):
    return X*Y - Y*X

In [19]:
x= comm(S2, chargeop)[0,3]

In [20]:
x

-I*n_01*(-E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 + omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha - epsilon^2_1 + 2*omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha - epsilon^1_1 + 2*omega_s)))/(-epsilon^1_1 - epsilon^2_1 + omega_s) - I*n_01*(E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 - omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha - epsilon^2_1 - 2*omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha - epsilon^1_1 - 2*omega_s)))/(-epsilon^1_1 - epsilon^2_1 - omega_s) - 

In [23]:
x

-I*n_01*(-E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 + omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha - epsilon^2_1 + 2*omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha - epsilon^1_1 + 2*omega_s)))/(-epsilon^1_1 - epsilon^2_1 + omega_s) - I*n_01*(E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 - omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha - epsilon^2_1 - 2*omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha - epsilon^1_1 - 2*omega_s)))/(-epsilon^1_1 - epsilon^2_1 - omega_s) - 

In [24]:
y = comm(S2, chargeop)[1,2]

In [25]:
y

-I*n_01*(-E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 + omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha - epsilon^2_1 + 2*omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(alpha + epsilon^1_1 + 2*omega_s)))/(epsilon^1_1 - epsilon^2_1 + omega_s) - I*n_01*(E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 - omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(epsilon^1_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha - epsilon^2_1 - 2*omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-alpha + epsilon^1_1 - 2*omega_s)))/(epsilon^1_1 - epsilon^2_1 - omega_s) - I*n_02

In [26]:
# from sympy import expand, collect
# # Define symbolic variables and operators
# lambda_sym = sp.Symbol('lambda')
# # A = sp.Symbol('A', commutative=False)
# # S1 = sp.Symbol('S1', commutative=False)
# # S2 = sp.Symbol('S2', commutative=False)
# # S3 = sp.Symbol('S3', commutative=False)

# # Define S up to third order in lambda
# S = lambda_sym * S1 + lambda_sym**2 * S2 
# # + lambda_sym**3 * S3


# # Perform BCH expansion
# result = chargeop + comm(S, chargeop) + (1/2) * comm(S, comm(S, chargeop))
# # + (1/6) * comm(S, comm(S, comm(S, A)))

# # Expand the result
# expanded_result = expand(result)

# # Collect terms by order of lambda
# collected = collect(expanded_result, lambda_sym, evaluate=False)

# first_order = collected.get(lambda_sym, 0)
# second_order = collected.get(lambda_sym**2, 0)
# # third_order = collected.get(lambda_sym**3, 0)
    

In [27]:
# second_order

In [28]:
chargeop_trans = sp.Rational(1,2)*chargeop*S1@S1 - chargeop*S2 - S1*chargeop*S1 + sp.Rational(1,2)*S1*S1*chargeop + S2*chargeop

In [33]:
# Create symbolic variables for epsilon1 and epsilon2, but set the first element to 0
x = chargeop_trans[0,3].subs(alpha,0)

In [94]:
epsilon1 = [0] + [sp.Symbol(f'epsilon^1_{i}') for i in range(1, d)]
epsilon2 = [0] + [sp.Symbol(f'epsilon^2_{i}') for i in range(1, d)]

In [96]:
x

I*E_L**2*n_12*phi^1_10*phi^2_10*phi_s01*phi_s02/(4*(-epsilon^1_1 + 2*omega_s)*(-epsilon^2_1 - omega_s)) - I*E_L**2*n_12*phi^1_10*phi^2_10*phi_s01*phi_s02/(4*(-epsilon^1_1 + omega_s)*(-epsilon^2_1 - 2*omega_s)) + I*E_L**2*n_12*phi^1_10*phi^2_10*phi_s01*phi_s02/(4*(-epsilon^1_1 - omega_s)*(-epsilon^2_1 + 2*omega_s)) - I*E_L**2*n_12*phi^1_10*phi^2_10*phi_s01*phi_s02/(4*(-epsilon^1_1 - 2*omega_s)*(-epsilon^2_1 + omega_s)) - I*n_01*(-E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 + omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^2_1 + 2*omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^1_1 + 2*omega_s)))/(-epsilon^1_1 - epsilon^2_1 + omega_s) - I*n_01*(E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 - omega_s)) + E_L**2*phi^

In [123]:
term11 = x.args[4]+x.args[6]
term12 = x.args[5]+x.args[7]
first_term = (term11 + term12).simplify()
first_term = ((first_term/first_term.args[-1]).subs(epsilon1[1],0).subs(epsilon2[1],0)*first_term.args[-1]).expand().subs(epsilon1[1],p-epsilon2[1]).expand()

# Keep only the linear term with respect to p
first_term = sp.collect(first_term, p).coeff(p)
first_term = first_term.subs(epsilon1[1], 0).subs(epsilon2[1], 0)
# # Simplify the term

# p = sp.Symbol('p')
# first_term = ((term11/term11.args[-1]).subs(epsilon1[1],0).subs(epsilon2[1],0)*term11.args[-1]).expand().subs(epsilon1[1],p-epsilon2[1]) 
# first_term.simplify()

In [125]:
first_term*p

I*E_L**2*n_12*p*phi^1_10*phi^2_10*phi_s01*phi_s02/(8*omega_s**3)

In [176]:
second_term = (x.args[0] + x.args[1]).expand()
simplified_args = [arg.factor() for arg in second_term.args]
simplified_second_term = sum(simplified_args)
second_term = simplified_second_term.subs(epsilon1[1],p-epsilon2[1]).subs(epsilon1[1],0).subs(epsilon2[1],0).simplify()
n,d = sp.fraction(second_term)
second_term = (n/d.subs(p,0)).expand()
second_term = sp.collect(second_term, p).coeff(p)*p


In [32]:
import pickle

# Store x in a file
with open('x_variable.pkl', 'wb') as file:
    pickle.dump(x, file)

# Store x in a file
with open('S2.pkl', 'wb') as file:
    pickle.dump(S2, file)



In [125]:
y = chargeop_trans[1,2]

In [126]:
x.free_symbols

{E_L,
 alpha,
 epsilon^1_1,
 epsilon^2_1,
 n_01,
 n_12,
 omega_s,
 phi^1_10,
 phi^2_10,
 phi_s00,
 phi_s01,
 phi_s02,
 phi_s11,
 phi_s12}

In [53]:
((x.args[3]+x.args[5]).simplify().args[-1].simplify()+(x.args[4]+x.args[2]).simplify().args[-1].simplify()).simplify()

2*omega_s*(-epsilon^1_1 - epsilon^2_1)

In [54]:
(x.args[4]+x.args[2]).simplify()

I*E_L**2*n_12*phi^1_10*phi^2_10*phi_s01*phi_s02*(-(epsilon^1_1 - omega_s)*(epsilon^2_1 + 2*omega_s) + (epsilon^1_1 + omega_s)*(epsilon^2_1 - 2*omega_s))/(4*(epsilon^1_1 - omega_s)*(epsilon^1_1 + omega_s)*(epsilon^2_1 - 2*omega_s)*(epsilon^2_1 + 2*omega_s))

In [57]:
result1 = sum(term / (-epsilon1[1] - epsilon2[1] + omega_s) for term in x.args[0].args[-1].args)
result1

-E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 + omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 + omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 + omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 + omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^2_1 + 2*omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^1_1 + 2*omega_s)*(-epsilon^1_1 - epsilon^2_1 + omega_s))

In [58]:
result2 = sum(term / (-epsilon1[1] - epsilon2[1] - omega_s) for term in x.args[1].args[-1].args)
result2

E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^2_1 - omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s)) + E_L**2*phi^1_10*phi^2_10*phi_s00*phi_s01/(4*(-epsilon^1_1 - omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^2_1 - omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s01*phi_s11/(4*(-epsilon^1_1 - omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^2_1 - 2*omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s)) - E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12/(4*(-epsilon^1_1 - 2*omega_s)*(-epsilon^1_1 - epsilon^2_1 - omega_s))

In [59]:
num1 = ((result2.args[0] + result2.args[4]).simplify()+(result1.args[0]+result1.args[2]).simplify()).simplify().args[-1].simplify()
num2 = ((result1.args[1]+result1.args[3]).simplify() + (result2.args[5] + result2.args[1]).simplify()).simplify().args[-1].simplify()
(num1 + num2).simplify()

6*omega_s*(-epsilon^1_1 - epsilon^2_1)

In [60]:
((result1.args[1]+result1.args[3]).simplify() + (result2.args[5] + result2.args[1]).simplify()).simplify()

E_L**2*phi^1_10*phi^2_10*phi_s01*(phi_s00 - phi_s11)*((epsilon^2_1 - omega_s)*(epsilon^1_1 + epsilon^2_1 - omega_s) - (epsilon^2_1 + omega_s)*(epsilon^1_1 + epsilon^2_1 + omega_s))/(4*(epsilon^2_1 - omega_s)*(epsilon^2_1 + omega_s)*(epsilon^1_1 + epsilon^2_1 - omega_s)*(epsilon^1_1 + epsilon^2_1 + omega_s))

In [61]:
num1  = (((result1.args[5]).simplify() + (result2.args[3]).simplify()).simplify()).simplify().args[-1].simplify()
num2 = (((result1.args[4]).simplify() + (result2.args[2]).simplify()).simplify()).simplify().args[-1].simplify()
(num1 + num2).simplify()

10*omega_s*(-epsilon^1_1 - epsilon^2_1)

In [62]:
(((result1.args[5]).simplify() + (result2.args[3]).simplify()).simplify()).simplify()

-E_L**2*phi^1_10*phi^2_10*phi_s02*phi_s12*((epsilon^2_1 - 2*omega_s)*(epsilon^1_1 + epsilon^2_1 - omega_s) - (epsilon^2_1 + 2*omega_s)*(epsilon^1_1 + epsilon^2_1 + omega_s))/(4*(epsilon^2_1 - 2*omega_s)*(epsilon^2_1 + 2*omega_s)*(epsilon^1_1 + epsilon^2_1 - omega_s)*(epsilon^1_1 + epsilon^2_1 + omega_s))

In [63]:
p = epsilon1[1] + epsilon2[1]
d = epsilon2[1] - epsilon1[1]
app1 = 3/2*p/omega_s**3
app2 = 5/8*p/omega_s**3
import numpy as np
app3 = -sp.sqrt(2)/8*p/omega_s**3

In [64]:
app3.subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06).simplify()/2

-0.000211892853546373*sqrt(2)

In [385]:
app1.subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06).simplify()

0.00508542848511295

In [387]:
app2.subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06).simplify()

0.00211892853546373

In [379]:
(x).subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06).simplify()

I*E_L**2*phi^1_10*phi^2_10*(-0.00254613175419209*phi_s00*phi_s01 - 0.000211974613308473*sqrt(2)*phi_s01*phi_s02 + 0.00254613175419209*phi_s01*phi_s11 + 0.001060663937768*phi_s02*phi_s12)/phi_s

In [163]:
app2.subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06)

-0.00494276795005203

In [366]:
((y).subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06).simplify()*5).simplify()

1.0*I*E_L**2*phi^1_10*phi^2_10*(-0.00239283250654304*phi_s00*phi_s01 - 0.000199611804414068*sqrt(2)*phi_s01*phi_s02 + 0.00239283250654304*phi_s01*phi_s11 + 0.000996658142921368*phi_s02*phi_s12)/phi_s

In [41]:
(x.args[0]+x.args[1]).args[0].args[4].subs(epsilon1[1],0.41).subs(omega_s,3.1).subs(epsilon2[1],0.6)

-1.963958635441*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/omega

In [42]:
(x.args[0]+x.args[1]).args[1].args[4].subs(epsilon1[1],0.41).subs(omega_s,3.1).subs(epsilon2[1],0.6)

-1.36716691157378*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/omega